In [2]:
import time
import random
import logging

import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from generate_dataset import *

In [5]:
class svm(object):
    def __init__(self, kernel='linear', epsilon=0.001):
        self.kernel = kernel
        self.epsilon = epsilon
    
    
    def _init_parameters(self, features, labels):
        
        self.X = features
        self.Y = labels
        
        self.b = 0.0
        self.n = len(features[0])
        self.N = len(features)
        self.alpha = [0.0] * self.N
        self.E = [self._E_(i) for i in range(self.N)]
        self.C = 1000
        self.max_interation = 5000
        
        
    def satisfy_kkt(self, i):
        
        ygx  = self.Y[i] * self._g_(i)
        if abs(self.slpha[i]) < self.epsilon:
            return ygx > 1 or ygx == 1
        elif abs(self.aplha[i] - self.C) < self.epsilon:
            return ygx < 1 or ygx == 1
        else:
            return abs(ygx - 1) < self.epsilon
    
    
    def is_top(self):
        
        for i in range(self.N):
            satisfy = self.satisfy_kkt(i)
            
            if not satisfy:
                return False
            
        return True
    
    
    def select_two_parameters(self):
        
        index_list = [i for i in range(self.N)]
        
        i1_list_1 = filter(lambda i:self.alpha[i] > 0 and self.alpha[i] < self.C, index_list)
        i1_list_2 = list(set(index_list) - set(i1_list_1))
        
        i1_list = i1_list_1
        i1_list.append(i1_list_2)
        
        for i in i1_list:
            
            if self.satisfy_kkt(i):
                continue
            
            E1 = self.E[i]
            max_ = (0,0)
            
            for j in index_list:
                if i == j:
                    continue
                
                E2 = self.E[j]
                if abs(E1 - E2) > max_[0]:
                    max_ = (abs(E1 - E2), j)
            
            return i, max_[1]
        
    def _K_(self, x1, x2):
            
            if self.kernel == 'linear':
                return sum(x1[k] * x2[k] for k in range(self.n))
            if self.kernel == 'poly':
                return (sum([x1[k] * x2[k] for k in range(self.n)]) + 1) **3
            
            print('无定义核函数')
            return 0

        
        
    def _g_(self, i):
            
            result = self.b
            for j in range(self.N):
                result += self.alpha[j] * self.Y[j] * self._K_(self.X[i], self.X[j])
            return result
        
        
    def _E_(self, i):
            
            return self._g_(i) - self.Y[i]
        
        
        def try_E(self, i):
            result = self.b - self.Y[i]
            for j in range(self.N):
                if self.alpha[j] < 0 or self.alpha[j]>self.C:
                    continue
                result += self.Y[j] * self.alpha[j] * self._K_(self.X[i], self.X[j])
            return result
        
        
        def train(self, features, labels):
            
            self._init_parameters(features, labels)
            
            for times in range(self.max_interation):
                
                logging.debug('iterater %d' % times)
                
                i1, i2 = self.select_two_parameters()
                
                L = max(0, self.alpha[i2] - self.alpha[i1])
                H = min(self.C, self.alpha[i1] + self.alpha[i2])
                
                if self.Y[i1] == self.Y[i2]:
                    L = max(0, self.alpha[i2] - self.alpha[i1] - self.C)
                    H = min(self.C, self.alpha[i1] + self.alpha[i2])
                
                E1 = self.E[i1]
                E2 = self.E[i2]
                eta = self._K_(self.X[i2] , self.X[i2] + self._K_(self.X[i2, self.X[i2]])
                               - 2*self._K_(self.X[i1], self.X[i2]))
                
                alpha2_new_unc = self.alpha[i2] + self.Y[i2] * (E1 - E2) / eta
                
                alph2_new = 0
                
                if alpha2_new_unc > H:
                    alph2_new = H
                elif alpha2_new_unc <L:
                    alph2_new = L
                else:
                    alph2_new = alpha2_new_unc
                
                alph1_new = self.alpha[i1] + self.Y[i1] *\
                            self.Y[i2] *(self.alpha[i2] - alph2_new)
                
                b_new = 0
                
                b1_new = -E1 - self.Y[i1] * self._K_(self.X[i1], self.X[i1])*(alph1_new - self.alpha[i1])\
                        - self.Y[i2]* self._K_(self.X[i2], self.X[i1])* (alph2_new _ alph1_new) + self.b
                b2_new = -E1 - self.Y[i1] * self._K_(self.X[i1], self.X[i2])*(alph1_new - self.alpha[i1])\
                        - self.Y[i2]* self._K_(self.X[i2], self.X[i2])* (alph2_new _ alph1_new) + self.b
                
                if alph1_new > 0 and alph2_new < self.C:
                    b_new = b1_new
                elif alph2_new > 0 and alph2_new < self.C:
                    b_new = b2_new
                else:
                    b_new = (b1_new + b2_new)/2
                    
                self.alpha[i1] = alph1_new
                self.alpha[i2] = alph2_new
                self.b = b_new
                
                self.E[i1] = self._E_(i1)
                self.E[i2] = self._E_(i2)
                
                
                
    def _predict_(self,feature):
        result = self.b

        for i in xrange(self.N):
            result += self.alpha[i]*self.Y[i]*self._K_(feature,self.X[i])

        if result > 0:
            return 1
        return -1

    def predict(self,features):
        results = []

        for feature in features:
            results.append(self._predict_(feature))

        return results